In [1]:
from __future__ import print_function, division
from gurobipy import Model, GRB, quicksum, LinExpr
import networkx as nx
import pickle
import numpy, random

In [2]:
def MIP_IM():
    budget = 25
    m = 100
    epsilon = 1
    
    attributes = ['gender','age', 'region', 'ethnicity']
  
    for attribute in attributes:
        for index in range(20):
            with open('../MIP/data/networks_prob/graph_spa_500_'+str(index)+'.pickle', "rb") as f:
                main_graph = pickle.load(f)

                samples = []
                for j in range(m):
                    G = nx.DiGraph()
                    for u in main_graph.nodes():
                        G.add_node(u)
                    for u,v in main_graph.edges():
                        if main_graph[u][v]['p']> random.random():
                            G.add_edge(u, v)
                    samples.append(G)


                model = Model('fairseed_'+str(index))
                min_value = model.addVar(lb=0.0, ub=1.0, vtype=GRB.CONTINUOUS)

                mvars = []
                #active nodes
                avars = []
                #seed nodes
                svars = []
                var_seed_dict = {}
                var_active_dict = {}
                var_mean_dict = {}

                for j in range(len(main_graph.nodes())):
                    s = model.addVar(lb=0.0, ub=1.0, vtype=GRB.BINARY)
                    svars.append(s)
                    var_seed_dict[j] = s

                for sample_index, sample in enumerate(samples):
                    for j in range(len(main_graph.nodes())):
                        a = model.addVar(lb=0.0, ub=1.0, vtype=GRB.BINARY)
                        avars.append(a)
                        var_active_dict[(sample_index,j)] = a    

                mvars.append(avars)
                mvars.append(svars)

                model.addConstr(quicksum(svars), GRB.LESS_EQUAL, budget)

                obj_expr = quicksum(avars)
                model.setObjective(min_value, GRB.MAXIMIZE)

                for sample_index, sample in enumerate(samples):
                    for i in range(len(main_graph.nodes())):
                        neighbors = sample.in_edges(i) 
                        e = len(neighbors)
                        ai = var_active_dict[(sample_index,i)]
                        si = var_seed_dict[i]
                        if i in  var_mean_dict.keys():
                            var_mean_dict[i]+= ai
                        else:
                            var_mean_dict[i]= ai
                        neighbors_vars = []
                        for neighbor in neighbors:
                            neighbors_vars.append(var_active_dict[(sample_index,neighbor[0])])
                        expr = (e+1)*ai-si-quicksum(neighbors_vars)
                        model.addConstr(expr, GRB.GREATER_EQUAL, 0)
                        model.addConstr(expr, GRB.LESS_EQUAL, e)

                labels = nx.get_node_attributes(main_graph, attribute)
                label_dict = {}
                for i in range(len(main_graph.nodes())):
                    label = labels[i].encode('utf-8')
                    if label in label_dict.keys():
                        label_dict[label].append(i)
                    else:
                        label_dict[label] = [i]     

                for label in label_dict.keys():
                    seed_vars = []
                    label_size = len(label_dict[label])
                    frac = float(label_size)/float(len(main_graph.nodes()))
                    for node in label_dict[label]:
                        seed_vars.append(var_seed_dict[node])
                    label_budget = budget*frac
                    print(label_budget)
                    expr = quicksum(seed_vars)
                    model.addConstr(expr, GRB.LESS_EQUAL, label_budget+ epsilon)
                    model.addConstr(expr, GRB.GREATER_EQUAL, label_budget-epsilon)    
                    

                try:
                    model.optimize()
                except e:
                    print(e)

                with open('../../../../Git/influence_maximization/experiments/im500/results/fairmip/fairseed/base100/'+attribute+'/output_'+str(index)+'.txt', "w") as of:    
                    for key,value in var_seed_dict.items():
                        if(value.x > 0):
                            print(key, file=of)

In [3]:
MIP_IM()

12.75
12.25
Optimize a model with 100005 rows, 50501 columns and 250126 nonzeros
Variable types: 1 continuous, 50500 integer (50500 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+01]
Presolve removed 63775 rows and 31901 columns
Presolve time: 1.34s
Presolved: 36230 rows, 18600 columns, 114020 nonzeros
Variable types: 0 continuous, 18600 integer (18600 binary)
Found heuristic solution: objective 1.0000000

Explored 0 nodes (0 simplex iterations) in 1.67 seconds
Thread count was 8 (of 8 available processors)

Solution count 1: 1 

Optimal solution found (tolerance 1.00e-04)
Best objective 1.000000000000e+00, best bound 1.000000000000e+00, gap 0.0000%
12.15
12.85
Optimize a model with 100005 rows, 50501 columns and 250004 nonzeros
Variable types: 1 continuous, 50500 integer (50500 binary)
Coefficient statistics:
  Matrix range     [1e+00, 6e+00]
  Objective range  [1e+00, 

Found heuristic solution: objective 1.0000000

Explored 0 nodes (0 simplex iterations) in 1.62 seconds
Thread count was 8 (of 8 available processors)

Solution count 1: 1 

Optimal solution found (tolerance 1.00e-04)
Best objective 1.000000000000e+00, best bound 1.000000000000e+00, gap 0.0000%
12.3
12.7
Optimize a model with 100005 rows, 50501 columns and 249732 nonzeros
Variable types: 1 continuous, 50500 integer (50500 binary)
Coefficient statistics:
  Matrix range     [1e+00, 6e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+01]
Presolve removed 64323 rows and 32182 columns
Presolve time: 1.26s
Presolved: 35682 rows, 18319 columns, 112005 nonzeros
Variable types: 0 continuous, 18319 integer (18319 binary)
Found heuristic solution: objective 1.0000000

Explored 0 nodes (0 simplex iterations) in 1.59 seconds
Thread count was 8 (of 8 available processors)

Solution count 1: 1 

Optimal solution found (tolerance 1.00e-04)
Best objec

Coefficient statistics:
  Matrix range     [1e+00, 6e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [5e-01, 2e+01]
Presolve removed 64698 rows and 32376 columns
Presolve time: 1.27s
Presolved: 35317 rows, 18125 columns, 111316 nonzeros
Variable types: 0 continuous, 18125 integer (18125 binary)
Found heuristic solution: objective 1.0000000

Explored 0 nodes (0 simplex iterations) in 1.60 seconds
Thread count was 8 (of 8 available processors)

Solution count 1: 1 

Optimal solution found (tolerance 1.00e-04)
Best objective 1.000000000000e+00, best bound 1.000000000000e+00, gap 0.0000%
3.4
1.35
2.35
2.75
5.55
5.95
3.65
Optimize a model with 100015 rows, 50501 columns and 250484 nonzeros
Variable types: 1 continuous, 50500 integer (50500 binary)
Coefficient statistics:
  Matrix range     [1e+00, 6e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [4e-01, 2e+01]
Presolve removed 63982 rows and 32011 colum

Variable types: 0 continuous, 18219 integer (18219 binary)
Found heuristic solution: objective 1.0000000

Explored 0 nodes (0 simplex iterations) in 1.62 seconds
Thread count was 8 (of 8 available processors)

Solution count 1: 1 

Optimal solution found (tolerance 1.00e-04)
Best objective 1.000000000000e+00, best bound 1.000000000000e+00, gap 0.0000%
2.5
1.85
1.95
2.4
6.25
6.2
3.85
Optimize a model with 100015 rows, 50501 columns and 249938 nonzeros
Variable types: 1 continuous, 50500 integer (50500 binary)
Coefficient statistics:
  Matrix range     [1e+00, 6e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [8e-01, 2e+01]
Presolve removed 64165 rows and 32106 columns
Presolve time: 1.30s
Presolved: 35850 rows, 18395 columns, 112861 nonzeros
Variable types: 0 continuous, 18395 integer (18395 binary)
Found heuristic solution: objective 1.0000000

Explored 0 nodes (0 simplex iterations) in 1.63 seconds
Thread count was 8 (of 8 available processo


Solution count 1: 1 

Optimal solution found (tolerance 1.00e-04)
Best objective 1.000000000000e+00, best bound 1.000000000000e+00, gap 0.0000%
10.55
0.35
0.9
0.25
0.55
0.75
0.1
0.1
0.25
0.4
0.15
9.9
0.75
Optimize a model with 100027 rows, 50501 columns and 250510 nonzeros
Variable types: 1 continuous, 50500 integer (50500 binary)
Coefficient statistics:
  Matrix range     [1e+00, 6e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e-01, 2e+01]
Presolve removed 67590 rows and 33590 columns
Presolve time: 1.25s
Presolved: 32437 rows, 16911 columns, 102428 nonzeros
Variable types: 0 continuous, 16911 integer (16911 binary)
Found heuristic solution: objective 1.0000000

Explored 0 nodes (0 simplex iterations) in 1.60 seconds
Thread count was 8 (of 8 available processors)

Solution count 1: 1 

Optimal solution found (tolerance 1.00e-04)
Best objective 1.000000000000e+00, best bound 1.000000000000e+00, gap 0.0000%
10.55
0.35
0.9
0.25
0.55
0.75
0

Thread count was 8 (of 8 available processors)

Solution count 1: 1 

Optimal solution found (tolerance 1.00e-04)
Best objective 1.000000000000e+00, best bound 1.000000000000e+00, gap 0.0000%
10.55
0.35
0.9
0.25
0.55
0.75
0.1
0.1
0.25
0.4
0.15
9.9
0.75
Optimize a model with 100027 rows, 50501 columns and 250196 nonzeros
Variable types: 1 continuous, 50500 integer (50500 binary)
Coefficient statistics:
  Matrix range     [1e+00, 6e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e-01, 2e+01]
Presolve removed 67563 rows and 33580 columns
Presolve time: 1.27s
Presolved: 32464 rows, 16921 columns, 102464 nonzeros
Variable types: 0 continuous, 16921 integer (16921 binary)
Found heuristic solution: objective 1.0000000

Explored 0 nodes (0 simplex iterations) in 1.62 seconds
Thread count was 8 (of 8 available processors)

Solution count 1: 1 

Optimal solution found (tolerance 1.00e-04)
Best objective 1.000000000000e+00, best bound 1.000000000000e+


Solution count 1: 1 

Optimal solution found (tolerance 1.00e-04)
Best objective 1.000000000000e+00, best bound 1.000000000000e+00, gap 0.0000%
11.8
7.75
3.5
1.0
0.95
Optimize a model with 100011 rows, 50501 columns and 250500 nonzeros
Variable types: 1 continuous, 50500 integer (50500 binary)
Coefficient statistics:
  Matrix range     [1e+00, 6e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [5e-02, 2e+01]
Presolve removed 64589 rows and 32291 columns
Presolve time: 1.30s
Presolved: 35422 rows, 18210 columns, 111515 nonzeros
Variable types: 0 continuous, 18210 integer (18210 binary)
Found heuristic solution: objective 1.0000000

Explored 0 nodes (0 simplex iterations) in 1.64 seconds
Thread count was 8 (of 8 available processors)

Solution count 1: 1 

Optimal solution found (tolerance 1.00e-04)
Best objective 1.000000000000e+00, best bound 1.000000000000e+00, gap 0.0000%
0.7
12.4
7.8
3.35
0.75
Optimize a model with 100011 rows, 50501 colum

  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e-01, 2e+01]
Presolve removed 64846 rows and 32415 columns
Presolve time: 1.31s
Presolved: 35165 rows, 18086 columns, 110865 nonzeros
Variable types: 0 continuous, 18086 integer (18086 binary)
Found heuristic solution: objective 1.0000000

Explored 0 nodes (0 simplex iterations) in 1.64 seconds
Thread count was 8 (of 8 available processors)

Solution count 1: 1 

Optimal solution found (tolerance 1.00e-04)
Best objective 1.000000000000e+00, best bound 1.000000000000e+00, gap 0.0000%
11.45
8.1
3.45
1.25
0.75
Optimize a model with 100011 rows, 50501 columns and 249992 nonzeros
Variable types: 1 continuous, 50500 integer (50500 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e-01, 2e+01]
Presolve removed 64229 rows and 32114 columns
Presolve time: 1.35s
Presolved: 35782 rows, 18387 columns, 11235